In [19]:
from gurobipy import *
import numpy as np
import math as mt


In [20]:
# Données
service_count=2
project_count=4
n = 2
p=4
budget_limit = 100
constraint_count=n*n +1
variable_count= n*n + n  + p
couts_projet = np.array([40, 50, 60, 50])

In [21]:
# Matrice d'utilte
matrice_utilite = np.array([[19, 6, 17, 2], [2, 11, 4, 18]])
print(matrice_utilite)

[[19  6 17  2]
 [ 2 11  4 18]]


In [22]:
# vecteur w
w = [2, 1]

# creer le vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])
print(w_prime)

[1, 1]


In [23]:
# matrice utilite
u = matrice_utilite

In [24]:
## les variables rk
coef_rk = []
for k in range(n):
        for i in range(n):
                coef_rk.append([])
                for j in range(n):
                        coef_rk[-1].append(0)
                coef_rk[-1][k] = 1
coef_rk = np.array(coef_rk)
print(coef_rk)

[[1 0]
 [1 0]
 [0 1]
 [0 1]]


In [25]:
## les variables bik
coef_bik = -1 * np.identity(n**2)
print(coef_bik)

[[-1. -0. -0. -0.]
 [-0. -1. -0. -0.]
 [-0. -0. -1. -0.]
 [-0. -0. -0. -1.]]


In [26]:
## les variables x
coef_xi = []
for k in range(n**2):
        coef_xi.append([])
        for i in range(p):
                coef_xi[-1].append(-u[k%n][i])
print(coef_xi)
coef_xi = np.array(coef_xi)

[[-19, -6, -17, -2], [-2, -11, -4, -18], [-19, -6, -17, -2], [-2, -11, -4, -18]]


In [27]:
# contrainte sur le budget
budget = [0 for i in range(n*(n+1))]
budget += [couts_projet[i] for i in range(p)]
budget = np.array(budget)
print(budget)

[ 0  0  0  0  0  0 40 50 60 50]


In [28]:
# concatenation des matrices rk, bik, xk 
result_concat = np.concatenate(
        (coef_rk, np.concatenate((coef_bik, coef_xi),axis=1)),
        axis=1)

# ajout de la contrainte sur les budgets
constrainte = np.vstack([result_concat, budget])
print(constrainte)

[[  1.   0.  -1.  -0.  -0.  -0. -19.  -6. -17.  -2.]
 [  1.   0.  -0.  -1.  -0.  -0.  -2. -11.  -4. -18.]
 [  0.   1.  -0.  -0.  -1.  -0. -19.  -6. -17.  -2.]
 [  0.   1.  -0.  -0.  -0.  -1.  -2. -11.  -4. -18.]
 [  0.   0.   0.   0.   0.   0.  40.  50.  60.  50.]]


In [29]:
## second memebre
second_membre = [0 for _ in range(n**2)]
second_membre.append(budget_limit)
second_membre = np.array(second_membre)
print (second_membre)

[  0   0   0   0 100]


In [30]:
# Coefficient de la fonction objectif
## vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])

## coefficient fonction objectif
c = [(i+1) * w_prime[i] for i in range(n)] # coef des r_k
# coef des b_ik
for i in range(n) :
    for j in range(n) :
        c.append(-1*w_prime[i])

c += [0 for i in range(p)] # ajout des coef de x
c = np.array(c)
print(c)

[ 1  2 -1 -1 -1 -1  0  0  0  0]


In [31]:
# Creer les models de la fonction objectif
m = Model("exercice1.3")

In [32]:
# declaration des variables de decisions

x = []
for i in range(n):
    x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
for i in range (n):
    for j in range(n):
        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((j+1,(i+1)))))
for j in range(p):
    x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d" % (j+1) ))

In [33]:
# maj du modele pour integrer les nouvelles variables
m.update()
print(x)

[<gurobi.Var r1>, <gurobi.Var r2>, <gurobi.Var b11>, <gurobi.Var b21>, <gurobi.Var b12>, <gurobi.Var b22>, <gurobi.Var x1>, <gurobi.Var x2>, <gurobi.Var x3>, <gurobi.Var x4>]


In [34]:
# definition de l'objectif
obj = LinExpr();
obj =0
for j in range(variable_count):
    obj += c[j] * x[j]

m.setObjective(obj,GRB.MAXIMIZE)

In [35]:
# Definition des contraintes
# Definition des contraintes
lignes = range(constraint_count)
colonnes = range(variable_count)
print (lignes, colonnes)
for i in lignes:
    m.addConstr(quicksum(constrainte[i][j]*x[j] for j in colonnes) <= second_membre[i], "Contrainte%d" % i)

range(0, 5) range(0, 10)


In [36]:
# Resolution
m.optimize()


print("")                
print('Solution optimale:')
mvars = m.getVars()
names = m.getAttr('VarName', mvars)
values = m.getAttr('X', mvars)
result = dict(zip(names, values))
for key in result:
    print(key,'=',result[key])
print("")
print('Valeur de la fonction objectif :', m.objVal)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 10 columns and 28 nonzeros
Model fingerprint: 0xcb05b678
Variable types: 6 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 5 rows, 10 columns, 28 nonzeros
Variable types: 6 continuous, 4 integer (4 binary)

Root relaxation: objective 6.660000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   66.60000    0    1   -0.00000   66.60000      -     -    0s
H    0     0                      